# Bayesian parameter estimation with stan

## Useful links

- General stan documentation: https://mc-stan.org/users/documentation/

- stan for cognitive science: https://cognitive-science-stan.github.io/

## What is stan

[Stan](https://mc-stan.org/) is a probabilistic programming language for statistical inference written in C++.

The Stan language is used to specify a Bayesian model and to estimate posterior distributions.

Stan was created by a development team consisting of 34 members that includes Andrew Gelman, Bob Carpenter, Matt Hoffman, and Daniel Lee.

There are multiple stan interfaces:

- CmdStan: command-line executable for the shell
- RStan: integration with the R software environment, maintained by Andrew Gelman and colleagues
- **PyStan: integration with the Python programming language**
- MatlabStan: integration with the MATLAB numerical computing environment
- Stan.jl: integration with the Julia programming language
- StataStan: integration with Stata

We are only going to focus on `pystan`. However, the same stan model code can be used acrossed different interfaces.

An explanation of the **algorithms** that make the parameter estimation possible goes beyond the scope of this workshop. I refer to the stan manual section https://mc-stan.org/docs/2_26/reference-manual/algorithms.html for a better overview. I do actually recommend having basic knowledge on sampling algorithms if you intend to use Bayesian parameter estimation in your master/PhD project. Knowing about the sampler and its tuning parameters could help you understand model's diagnostics, failed convergence, failed intialization, etc. Stan uses two Markov chain Monte Carlo (MCMC) algorithms :the Hamiltonian Monte Carlo (HMC) algorithm and its adaptive variant the no-U-turn sampler (NUTS).

**Note**:

There are several methods for Bayesian parameter estimation, implemented in different languages.

An excellent alternative to `stan` in Python is for example [PyMC3](https://docs.pymc.io/), which also uses the NUTS sampler and does not require to learn stan language.

Otherwise `JAGS` is an alternative to `stan`, and interfaces to `JAGS` are available in Python, R, and matlab as well.

## Stan language



### Data Types and Declarations

As in C++ (and unlike in Python), every variable used in a Stan program must have a **declared data type**. Only values of that type will be assignable to that variable. Distributions can also be strict in the type of data they accept.

For example:
```
real x[10];
matrix[3, 3] m[6, 7];
```
Specify a 1D array `x` of size 10 made of continuous, unconstrained data, and a 2D array `m` of size 6x7, whose elements are matrices of size 3x3 (also continuous and unconstrained).

We can add constraints using `<>`:
```
int<lower = 1> N;
real<upper = 0> log_p;
vector<lower = -1, upper = 1>[3] rho;
```
Constraints are important as they specify ranges of data and parameter values for which the density is not zero. Otherwise, the samplers and optimizers may have any of a number of pathologies including just getting stuck, failure to initialize, excessive Metropolis rejection, or biased draws due to inability to explore the tails of the distribution.

```
int<lower=0,upper=1> cond;
real<lower=0> sigma;
```
**QUESTION**: What values can the variable `cond` and `sigma` take?

### Vectors, matrices, and arrays

These are the three main types of container objects:
- Vectors are intrinsically one-dimensional (real only)
- Matrices are intrinsically two dimensional (real only)
- Arrays can have N dimensions (specify whether int or real)

They are **indexed starting from one**. This follows the convention in statistics and linear algebra as well as their implementations in the statistical software packages R, MATLAB, BUGS, and JAGS. General computer programming languages, on the other hand, such as C++ and Python, index arrays starting from zero.

#### Vectors and matrices 
They are mostly used for:
- matrix arithmetic operations (e.g., matrix multiplication)
- linear algebra functions (e.g., eigenvalues and determinants)
- multivariate function parameters and outcomes (e.g., multivariate normal distribution arguments).

Vectors in Stan are column vectors. This is good to know for calculations and when assigning vectors to matrices' rows.
If you want to specify a row vector, you should use `row_vector`:
```
vector[4] u;
row_vector[7] s;
matrix[4, 7] a;

// ...
a[1] = s;
```
If v is a column vector or row vector, then v[2] is the second element in the vector. 

If m is a matrix, m[2] is the second row and m[2, 3] or m[2][3] are the value in the second row and third column.

#### Arrays
```
int n[5];
real a[3, 4];
real<lower=0> z[5, 4, 2];
vector[7] mu[3];
```
**QUESTION**: What is `mu`?

More on indexing: https://mc-stan.org/docs/2_26/reference-manual/language-indexing-section.html

## Program Blocks

A Stan program is organized into a sequence of named blocks, the bodies of which consist of **variable declarations**, followed in the case of some blocks with **statements**, executed in the order in which they are written. Variables must be defined to have some value (as well as declared to have some type) before they are used — if they do not, the behavior is undefined.

```
functions {
  // ... function declarations and definitions ...
}
data {
  // ... declarations ...
}
transformed data {
   // ... declarations ... statements ...
}
parameters {
   // ... declarations ...
}
transformed parameters {
   // ... declarations ... statements ...
}
model {
   // ... declarations ... statements ...
}
generated quantities {
   // ... declarations ... statements ...
}
```

The essential blocks for parameter estimation are data, parameters, model, and generated quantities:
- The function-definition block contains user-defined functions. 
- The data block declares the required data for the model. 
- The transformed data block allows the definition of constants and transforms of the data. 
- The parameters block declares the model’s parameters — the unconstrained version of the parameters is what’s sampled or optimized.
- The transformed parameters block allows variables to be defined in terms of data and parameters that may be used later and will be saved. 
- The model block is where the log probability function is defined. 
- The generated quantities block allows derived quantities based on parameters, data, and optionally (pseudo) random number generation.

The variables declared in each block have scope over all **subsequent** statements. 
Exceptions:
- variables declared in the model block cannot be accessed in the generated quantities block; to make a variable accessible in the model and generated quantities block, it must be declared as a transformed parameter.
- variables declared as function parameters have scope only within that function definition’s body

### Comments

You can add line-based comments anywhere whitespace is allowed in a Stan program:

```
data {
  int<lower=0> N;  // number of observations
  real y[N];  // observations
}
```

## Examples

Simple linear regression example (no prior specified, no posterior predictives):

```
data {
    int<lower=0> N;
    vector[N] x;
    vector[N] y;
}
parameters {
    real alpha;
    real beta;
    real<lower=0> sigma;
}
model {
    y ~ normal(alpha + beta * x, sigma);
}
```

Logistic regression example (prior specified, posterior predictives on test data):

```
data {
  int<lower = 1> N_train;
  vector[N_train] x_train;
  int<lower = 0, upper = 1> y_train[N_train];
  int<lower = 1> N_test;
  vector[N_test] x_test;
}
parameters {
  real alpha;
  real beta;
}
model {
  y_train ~ bernoulli_logit(alpha + beta*x_train);
  alpha ~ normal(5, 10);
  beta ~ normal(5, 10);
}
generated quantities {
  vector[N_test] y_test;
  for(i in 1:N_test) {
    y_test[i] = bernoulli_rng(inv_logit(alpha + beta*x_test[i]));
  }
}
```